## 1) Generate Sudoku - optimized trial layout

In [ ]:
using SudokuPlantDesign
using BenchmarkTools
using BlockArrays
using DataFrames
using Random
using XLSX
using PyPlot
using ColorTypes
using PyCall
@pyimport matplotlib.colors as matcolors

As explained in `sudoku_basic`, we generate a new configuration `conf` which is divided into 2 horizontal and 5 vertical blocks of dimensions `10` x `2` respectively. In total, there are `4` different checks in the configuration. All fields which are neither checks nor missing plots are defined as entries - these are the unreplicated genotypes which are augmented by replicated checks. We leave the plot `[1,3]` and `[1,4]` empty. Configuration is plotted again to show the update. Then, checks are initialized randomly with one check per check type per block.

In [ ]:
conf = get_configuration(
    [10,10],  [2,2,2,2,2],    4
    ;
    bc = :open
);

empty_plots!(conf, 1:1,3:4)
initialize_checks_per_block!(conf)
show_configuration(conf, zoom=0.2)
savefig("output/initial_random_check_distribution.pdf")

We use a standard selection of cost functions, as used in the `sudoku_basic` example.

In [ ]:
function K_indiv(conf :: C) :: Float64 where {C <: CheckConfiguration}
    return  K_num_checks_equal_per_type(conf) +
            K_checks_per_type_per_block(conf, 1)*20+
            K_neighbors_different_check_functional(conf, d->0.5/(d^3)) +
            K_neighbors_same_check_functional(conf, d->1/(d^3))
end

In the Sudoku-augmented design, we use 3 updates since we need to insert new checks, swap checks with each other and swap also checks with entries.

In [ ]:
costs = optimize_design!(
    conf,
    [UpdateNewCheckLabel(),UpdateSwapCheckCheck(),UpdateSwapCheckGenotype()],
    K_indiv
)

By inspecting the final plot, we see that the check distribution is optimized.

In [ ]:
costs .-= minimum(costs)
costs .+= 1

#print things
println("costs: ",K_indiv(conf)+costs[1], " -> ", K_indiv(conf)+costs[end])
println("perc_checks: ",conf.num_checks_total / (conf.num_plots_total - conf.num_checks_total))
println("entries: ",conf.num_plots_total - conf.num_checks_total)
println("checks:    ",conf.num_checks_total, "  (", round(100*conf.num_checks_total / conf.num_plots_total, digits=2), "%)")
for i in 1:conf.N
    println("check ($(i)): ",conf.num_checks[i], "  (", round(100*conf.num_checks[i] / conf.num_checks_total, digits=2), "%)")    
end


figure()
plot(costs)
yscale("log")


#save without check labels
show_configuration(conf,check_labels=false)
savefig("output/final_sudoku_design.pdf")

## 6) Plot with Plant_Codes and write data to xlsx

## 2) Save design data with field plan

First, we read in the input data `input_augmented.xlsx` with both sheets `checks` and `entries`. Each sheet contains the name of the genotype in the first column, and in following columns further properties of the genotypes can be added to be transferred to the final output file and field plan. 

In [ ]:
checkdata = DataFrame(XLSX.readtable("input_augmented.xlsx", "checks")...)

checknames=names(checkdata)
for i in 1:length(checknames)
    checkdata[!,Symbol(checknames[i])]=string.(collect(checkdata[!,Symbol(checknames[i])]))
end

for col in eachcol(checkdata)
    replace!(col,missing => "NA")
end


genotypedata = DataFrame(XLSX.readtable(templatename, "entries")...)
genonames=names(genotypedata)
for i in 1:length(checknames)
    genotypedata[!,Symbol(genonames[i])]=string.(collect(genotypedata[!,Symbol(genonames[i])]))
end

for col in eachcol(genotypedata)
    replace!(col,missing => "NA")
end

Since we usually want the entries to be randomized, their data is internally shuffled.

In [ ]:
genotypedata = genotypedata[shuffle(1:nrow(genotypedata)), :]
genotypedata[!,:genotype]=string.(collect(genotypedata[!,:genotype]))

Finally, we plot the final field map of the optimized Sudoku-augmented design and add position, block number and the labels to the field map.

In [ ]:
POS = get_POS_coordinatewise(C,"x","left","lower","snake") #with robot
L = get_Plant_Code_Label_allinfo(C,POS,genotypedata,checkdata)
show_pos_and_labels_allinfo(C, POS, L, zoom=0.9,fontsize=7,alpha=0.5,cmap=cmap_sudoku)
savefig("final_sudoku_design_labels_"*trialname*".pdf")

In [ ]:
The data table of the optimized Sudoku-augmented design can be saved.

In [ ]:
#write file to xlsx
write_to_xlsx_allinfo(C.configuration,trialname,POS,L)

In [ ]:
To be able to load the design again, we can save the configuration.

In [ ]:
#load_configuration(C, "configuration_"*trialname*".txt")
save_configuration(C, "configuration_"*trialname*".txt")